In [2]:
import os, time, sys
from pathlib import Path

SESSION_NAME = "DS1_Alte_Nationalgalerie"

ROOT = Path().absolute().parent
RAW_INPUT_PATH = Path(f"{ROOT}/data/1_raw_input")
INPUT_PATH = Path(f"{ROOT}/data/2_input")
RESULTS_PATH = Path(f"{ROOT}/data/3_results")

SESSION_ID = SESSION_NAME if SESSION_NAME else str(int(time.time()))
SESSION_PATH = Path(f"{RESULTS_PATH}/{SESSION_ID}")

!mkdir -p {RAW_INPUT_PATH}
!mkdir -p {INPUT_PATH}
!mkdir -p {SESSION_PATH}

print(f"Created new session with ID {SESSION_ID} under {RESULTS_PATH}")

Created new session with ID DS1_Alte_Nationalgalerie under /mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results


In [ ]:
##### Data Preparation ( raw input -> input folder)
%cd {ROOT}

raw_input_files = [x for x in RAW_INPUT_PATH.glob('**/*') if x.is_file()]
print(f"found {len(raw_input_files)} raw input files")
_f = "/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/1_raw_input/lego.mp4" #str(raw_input_files[0])
print(_f)
DOWNSAMPLING_RATE = 2

#!bash scripts/third_party/neuralangelo/run_ffmpeg.sh {{SESSION_NAME}} {{_f}} 2
! ffmpeg -i {_f} -vf "select=not(mod(n\,{DOWNSAMPLING_RATE}))" -vsync vfr -q:v 2 {INPUT_PATH}/%06d.jpg

In [3]:
# SPARSE RECONSTRUCTION: Step 1/3
import pycolmap

COLMAP_SFM_PATH = Path(f"{SESSION_PATH}/sparse")
COLMAP_MVS_PATH = Path(f"{SESSION_PATH}/dense")
COLMAP_DB_PATH = Path(f"{SESSION_PATH}/database.db")

!mkdir -p {COLMAP_MVS_PATH}
!mkdir -p {COLMAP_SFM_PATH}

pycolmap.extract_features(COLMAP_DB_PATH, INPUT_PATH)

In [4]:
# SPARSE RECONSTRUCTION: Step 2/3
pycolmap.match_exhaustive(COLMAP_DB_PATH)

I20240124 11:34:33.366691   420 misc.cc:198] 
Exhaustive feature matching
I20240124 11:34:34.510741   420 feature_matching.cc:231] Matching block [1/4, 1/4]
I20240124 11:34:35.145939   420 feature_matching.cc:46]  in 0.635s
I20240124 11:34:35.146376   420 feature_matching.cc:231] Matching block [1/4, 2/4]
I20240124 11:34:36.025260   420 feature_matching.cc:46]  in 0.879s
I20240124 11:34:36.025740   420 feature_matching.cc:231] Matching block [1/4, 3/4]
I20240124 11:34:36.499053   420 feature_matching.cc:46]  in 0.473s
I20240124 11:34:36.499544   420 feature_matching.cc:231] Matching block [1/4, 4/4]
I20240124 11:34:36.502789   420 feature_matching.cc:46]  in 0.003s
I20240124 11:34:36.502971   420 feature_matching.cc:231] Matching block [2/4, 1/4]
I20240124 11:34:36.852332   420 feature_matching.cc:46]  in 0.349s
I20240124 11:34:36.852782   420 feature_matching.cc:231] Matching block [2/4, 2/4]
I20240124 11:34:38.373593   420 feature_matching.cc:46]  in 1.521s
I20240124 11:34:38.374066 

In [5]:
# SPARSE RECONSTRUCTION: Step 3/3
maps = pycolmap.incremental_mapping(COLMAP_DB_PATH, INPUT_PATH, COLMAP_SFM_PATH)
maps[0].write(COLMAP_SFM_PATH)

I20240124 12:27:56.216686   593 misc.cc:198] 
Loading database
I20240124 12:27:56.264849   593 database_cache.cc:54] Loading cameras...
I20240124 12:27:56.266739   593 database_cache.cc:64]  1 in 0.002s
I20240124 12:27:56.266786   593 database_cache.cc:72] Loading matches...
I20240124 12:27:58.451627   593 database_cache.cc:78]  7724 in 2.185s
I20240124 12:27:58.451691   593 database_cache.cc:94] Loading images...
I20240124 12:27:59.481914   593 database_cache.cc:143]  159 in 1.030s (connected 159)
I20240124 12:27:59.481994   593 database_cache.cc:154] Building correspondence graph...
I20240124 12:27:59.892050   593 database_cache.cc:190]  in 0.410s (ignored 0)
I20240124 12:27:59.892980   593 timer.cc:91] Elapsed time: 0.060 [minutes]
I20240124 12:27:59.924502   593 misc.cc:198] 
Finding good initial image pair
I20240124 12:28:00.417174   593 misc.cc:198] 
Initializing with image pair #76 and #83
I20240124 12:28:00.424366   593 misc.cc:198] 
Global bundle adjustment
I20240124 12:28:00.

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  9.388352e+02    0.00e+00    1.11e+05   0.00e+00   0.00e+00  1.00e+04        0    2.82e-03    7.89e-03
   1  1.170986e+02    8.22e+02    4.03e+03   2.64e+00   1.00e+00  3.00e+04        1    4.23e-03    1.22e-02
   2  1.163288e+02    7.70e-01    7.58e+01   3.33e-01   1.00e+00  9.00e+04        1    2.84e-03    1.50e-02
   3  1.162823e+02    4.64e-02    9.58e+01   3.25e-01   9.82e-01  2.70e+05        1    2.78e-03    1.78e-02
   4  1.162486e+02    3.37e-02    1.34e+03   3.23e+00   6.62e-01  2.79e+05        1    2.76e-03    2.06e-02
   5  1.162130e+02    3.56e-02    1.20e+03   3.53e+00   7.62e-01  3.26e+05        1    2.78e-03    2.34e-02
   6  1.161889e+02    2.40e-02    1.16e+03   3.63e+00   7.07e-01  3.51e+05        1    2.79e-03    2.62e-02
   7  1.161686e+02    2.03e-02    8.90e+02   3.22e+00   7.77e-01  4.23e+05        1    2.74e-03    2.90e-02
   8  1.161553e+02    1.34e-

I20240124 12:28:00.632177   593 incremental_mapper.cc:43] => Added observations: 4229
I20240124 12:28:00.817520   593 misc.cc:205] 
Bundle adjustment report
------------------------
I20240124 12:28:00.817579   593 bundle_adjustment.cc:942] 
    Residuals : 14424
   Parameters : 9175
   Iterations : 16
         Time : 0.176307 [s]
 Initial cost : 0.738539 [px]
   Final cost : 0.330201 [px]
  Termination : Convergence

I20240124 12:28:00.827710   593 incremental_mapper.cc:78] => Merged observations: 4
I20240124 12:28:00.827762   593 incremental_mapper.cc:79] => Completed observations: 87
I20240124 12:28:00.827770   593 incremental_mapper.cc:81] => Filtered observations: 22
I20240124 12:28:00.827775   593 incremental_mapper.cc:90] => Changed observations: 0.015668
I20240124 12:28:00.894097   593 misc.cc:205] 
Bundle adjustment report
------------------------
I20240124 12:28:00.894160   593 bundle_adjustment.cc:942] 
    Residuals : 14554
   Parameters : 9148
   Iterations : 5
         Tim

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.095131e+03    0.00e+00    3.41e+03   0.00e+00   0.00e+00  1.00e+04        0    6.41e-03    1.51e-02
   1  2.078536e+03    1.66e+01    1.13e+01   1.03e+00   1.00e+00  3.00e+04        1    1.17e-02    2.69e-02
   2  2.078515e+03    2.17e-02    4.31e+00   5.54e-01   1.00e+00  9.00e+04        1    1.09e-02    3.79e-02
   3  2.078512e+03    2.13e-03    7.90e-01   2.19e-01   1.01e+00  2.70e+05        1    1.09e-02    4.88e-02
   4  2.078512e+03    4.19e-05    9.20e-01   3.27e-02   1.02e+00  8.10e+05        1    1.09e-02    5.97e-02
   5  2.078512e+03    1.90e-07    7.40e-02   1.99e-03   1.04e+00  2.43e+06        1    1.09e-02    7.07e-02
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.110138e+03    0.00e+00    1.98e+03   0.00e+00   0.00e+00  1.00e+04        0    6.53e-03    1.52e-02
   1  2.086431e+03    2.37e+

I20240124 12:28:01.144763   593 misc.cc:198] 
Registering image #88 (4)
I20240124 12:28:01.144807   593 incremental_mapper.cc:495] => Image sees 2073 / 9682 points
I20240124 12:28:01.232092   593 misc.cc:205] 
Pose refinement report
----------------------
I20240124 12:28:01.232146   593 bundle_adjustment.cc:942] 
    Residuals : 4146
   Parameters : 6
   Iterations : 46
         Time : 0.081367 [s]
 Initial cost : 0.753531 [px]
   Final cost : 0.710824 [px]
  Termination : Convergence

I20240124 12:28:01.237056   593 incremental_mapper.cc:40] => Continued observations: 2072
I20240124 12:28:01.250520   593 incremental_mapper.cc:43] => Added observations: 4587
I20240124 12:28:01.729015   593 misc.cc:205] 
Bundle adjustment report
------------------------
I20240124 12:28:01.729079   593 bundle_adjustment.cc:942] 
    Residuals : 27908
   Parameters : 15661
   Iterations : 19
         Time : 0.466825 [s]
 Initial cost : 0.605562 [px]
   Final cost : 0.442655 [px]
  Termination : Convergenc

...

I20240124 12:36:49.111486   593 misc.cc:205] 
Bundle adjustment report
------------------------
I20240124 12:36:49.111554   593 bundle_adjustment.cc:942] 
    Residuals : 1344436
   Parameters : 304048
   Iterations : 5
         Time : 9.63315 [s]
 Initial cost : 0.767656 [px]
   Final cost : 0.767408 [px]
  Termination : Convergence

I20240124 12:36:49.575244   593 incremental_mapper.cc:175] => Completed observations: 32
I20240124 12:36:49.764010   593 incremental_mapper.cc:178] => Merged observations: 88
I20240124 12:36:49.858965   593 incremental_mapper.cc:160] => Filtered observations: 38
I20240124 12:36:49.859093   593 incremental_mapper.cc:119] => Changed observations: 0.000235
I20240124 12:36:49.859462   593 incremental_mapper.cc:167] => Filtered images: 0
I20240124 12:36:49.980787   593 misc.cc:198] 
Registering image #144 (151)
I20240124 12:36:49.980850   593 incremental_mapper.cc:495] => Image sees 1002 / 3932 points
I20240124 12:36:50.008560   593 misc.cc:205] 
Pose refineme

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  8.870722e+05    0.00e+00    6.29e+05   0.00e+00   0.00e+00  1.00e+04        0    4.41e-01    2.37e+00
   1  8.439138e+05    4.32e+04    2.13e+05   6.64e+00   1.01e+00  3.00e+04        1    2.63e+00    5.01e+00
   2  8.430211e+05    8.93e+02    1.10e+05   4.02e+00   1.01e+00  9.00e+04        1    1.47e+00    6.48e+00
   3  8.429892e+05    3.19e+01    9.26e+03   1.82e+00   1.01e+00  2.70e+05        1    1.45e+00    7.93e+00
   4  8.429886e+05    5.97e-01    1.64e+02   3.22e-01   1.02e+00  8.10e+05        1    1.44e+00    9.37e+00
   5  8.429886e+05    1.12e-02    5.26e+00   3.09e-02   1.01e+00  2.43e+06        1    1.53e+00    1.09e+01
   6  8.429886e+05    7.43e-05    1.44e+00   2.25e-03   1.02e+00  7.29e+06        1    1.54e+00    1.24e+01
   7  8.429886e+05    1.96e-07    1.18e-01   1.02e-04   1.16e+00  2.19e+07        1    1.51e+00    1.40e+01


I20240124 12:37:10.523484   593 misc.cc:205] 
Bundle adjustment report
------------------------
I20240124 12:37:10.523548   593 bundle_adjustment.cc:942] 
    Residuals : 1363482
   Parameters : 309325
   Iterations : 8
         Time : 13.9941 [s]
 Initial cost : 0.806594 [px]
   Final cost : 0.786296 [px]
  Termination : Convergence

I20240124 12:37:10.996202   593 incremental_mapper.cc:175] => Completed observations: 159
I20240124 12:37:11.190163   593 incremental_mapper.cc:178] => Merged observations: 271
I20240124 12:37:11.287307   593 incremental_mapper.cc:160] => Filtered observations: 406
I20240124 12:37:11.287366   593 incremental_mapper.cc:119] => Changed observations: 0.001226
I20240124 12:37:11.287390   593 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  8.049758e+05    0.00e+00    3.08e+05   0.00e+00   0.00e+00  1.00e+04        0    4.53e-01    2.35e+00
   1  8.024471e+05    2.53e+03    7.59e+04   2.92e+00   9.98e-01  3.00e+04        1    2.77e+00    5.11e+00
   2  8.023034e+05    1.44e+02    4.51e+04   4.60e-01   9.93e-01  9.00e+04        1    1.52e+00    6.63e+00
   3  8.022885e+05    1.49e+01    6.05e+03   1.15e-01   9.99e-01  2.70e+05        1    1.52e+00    8.15e+00
   4  8.022881e+05    3.77e-01    1.73e+02   5.60e-02   1.01e+00  8.10e+05        1    1.50e+00    9.65e+00
   5  8.022881e+05    5.27e-03    3.61e+00   1.34e-02   1.02e+00  2.43e+06        1    1.46e+00    1.11e+01
   6  8.022881e+05    3.19e-05    1.24e+00   1.21e-03   1.03e+00  7.29e+06        1    1.51e+00    1.26e+01
   7  8.022881e+05    7.92e-08    1.08e-01   6.48e-05   9.38e-01  2.19e+07        1    1.53e+00    1.41e+01


I20240124 12:37:26.304227   593 misc.cc:205] 
Bundle adjustment report
------------------------
I20240124 12:37:26.304287   593 bundle_adjustment.cc:942] 
    Residuals : 1362980
   Parameters : 309151
   Iterations : 8
         Time : 14.1934 [s]
 Initial cost : 0.768505 [px]
   Final cost : 0.767221 [px]
  Termination : Convergence

I20240124 12:37:26.771273   593 incremental_mapper.cc:175] => Completed observations: 71
I20240124 12:37:26.960688   593 incremental_mapper.cc:178] => Merged observations: 52
I20240124 12:37:27.054550   593 incremental_mapper.cc:160] => Filtered observations: 88
I20240124 12:37:27.054600   593 incremental_mapper.cc:119] => Changed observations: 0.000310
I20240124 12:37:27.054855   593 incremental_mapper.cc:167] => Filtered images: 0
I20240124 12:37:27.069128   593 timer.cc:91] Elapsed time: 9.514 [minutes]


In [ ]:
# DENSE RECONSTRUCTION: Step 1/3
pycolmap.undistort_images(COLMAP_MVS_PATH, COLMAP_SFM_PATH, INPUT_PATH)

In [ ]:
# DENSE RECONSTRUCTION: Step 3/3
pycolmap.patch_match_stereo(COLMAP_MVS_PATH)  # requires compilation with CUDA

In [ ]:
# DENSE RECONSTRUCTION: Step 3/3
pycolmap.stereo_fusion(COLMAP_MVS_PATH / "dense.ply", COLMAP_MVS_PATH)

In [9]:
import sys

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print(sys.path)

generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json_advanced.py"
! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor" --image_dir "{INPUT_PATH}"

#generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json.py"
#! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor"

['/mnt/d/dev/python/historical-photo-sfm-pipeline/scripts', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python39.zip', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9/lib-dynload', '', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9/site-packages', '/mnt/d/dev/python/historical-photo-sfm-pipeline']
/mnt/d/dev/python/historical-photo-sfm-pipeline
Fraction of images looking at the center: 0.21.
Fraction of images positioned around the center: 0.75.
Valid fraction of concentric images: 0.17.
{1: Camera(id=1, model='SIMPLE_RADIAL', width=1536, height=2048, params=array([1.55216807e+03, 7.68000000e+02, 1.02400000e+03, 2.03460123e-02]))}
Writing data to json file:  /mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results/DS1_Alte_Nationalgalerie/transforms_withpoints.json


In [4]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif_data(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()

        if exif_data is not None:
            # Decode the EXIF data
            decoded_exif = {TAGS[key]: exif_data[key] for key in exif_data.keys() if key in TAGS and isinstance(exif_data[key], (int, str, bytes))}
            return decoded_exif
        else:
            print("No EXIF data found.")
            return None

    except Exception as e:
        print(f"Error reading EXIF data: {e}")
        return None

# Example usage
image_path = '/mnt/d/dev/python/historical-photo-sfm-pipeline/data/2_input/IMG_0555.jpeg'
exif_data = get_exif_data(image_path)

if exif_data:
    print("EXIF Metadata:")
    for key, value in exif_data.items():
        print(f"{key}: {value}")

EXIF Metadata:
ResolutionUnit: 2
ExifOffset: 232
Make: Apple
Model: iPhone 11 Pro Max
Software: 17.2.1
DateTime: 2024:01:19 15:50:17
YCbCrPositioning: 1
HostComputer: iPhone 11 Pro Max
ExifVersion: b'0232'
ComponentsConfiguration: b'\x01\x02\x03\x00'
DateTimeOriginal: 2024:01:19 15:50:17
DateTimeDigitized: 2024:01:19 15:50:17
MeteringMode: 5
Flash: 16
ColorSpace: 65535
ExifImageWidth: 1536
FocalLengthIn35mmFilm: 26
SceneCaptureType: 0
OffsetTime: +01:00
OffsetTimeOriginal: +01:00
OffsetTimeDigitized: +01:00
SubsecTimeOriginal: 660
SubsecTimeDigitized: 660
ExifImageHeight: 2048
SensingMethod: 2
SceneType: b'\x01'
ExposureProgram: 2
ISOSpeedRatings: 32
ExposureMode: 0
FlashPixVersion: b'0100'
WhiteBalance: 0
LensMake: Apple
LensModel: iPhone 11 Pro Max back triple camera 4.25mm f/1.8
CompositeImage: 2
MakerNote: b'Apple iOS\x00\x00\x01MM\x00+\x00\x01\x00\t\x00\x00\x00\x01\x00\x00\x00\x0e\x00\x02\x00\x07\x00\x00\x02\x00\x00\x00\x02\x18\x00\x03\x00\x07\x00\x00\x00h\x00\x00\x04\x18\x00\x04\

In [ ]:
'''
camera = pycolmap.Camera(
    model=camera_model_name_or_id,
    width=width,
    height=height,
    params=params,
)

import pycolmap
reconstruction = pycolmap.Reconstruction("path/to/reconstruction/dir")
print(reconstruction.summary())

for image_id, image in reconstruction.images.items():
    print(image_id, image)

for point3D_id, point3D in reconstruction.points3D.items():
    print(point3D_id, point3D)

for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

reconstruction.write("path/to/reconstruction/dir/")
'''